# シミュレーションコード

処理の整理
- 遷移確率の算出・正解データの作成
    - 期間を定義し、parquet fileから各時間帯の遷移確率を算出する
    - データリークが起きないように期間を設定、正解データとする
- シミュレーション
    - シミュレーションパラメータの設定
        - mesh レベルの設定
        - 初期配置のPoIの設定
        - 開始時間
        - 終了時間
        - 遷移分岐の最大数
        - トップN設定



## ライブラリ

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd


## 正解データ・遷移確率の算出

In [5]:
from pathlib import Path

BASE_DIR = Path("../../data/processed/yamada_processed/trans_count")

parquet_paths = sorted(BASE_DIR.glob("*.parquet"))
print("found:", len(parquet_paths))

renamed = []
skipped = []

for p in parquet_paths:
    new_path = p.with_suffix(".csv")

    # すでに同名csvがある場合は事故るのでスキップ
    if new_path.exists():
        skipped.append((p, "csv already exists"))
        continue

    p.rename(new_path)
    renamed.append((p, new_path))

print("renamed:", len(renamed))
print("skipped:", len(skipped))

for old, new in renamed[:10]:
    print("OK:", old.name, "->", new.name)

if skipped:
    print("\n[SKIPPED]")
    for p, reason in skipped[:10]:
        print("SKIP:", p.name, ":", reason)


found: 14
renamed: 14
skipped: 0
OK: state_count_20190201.parquet -> state_count_20190201.csv
OK: state_count_20190202.parquet -> state_count_20190202.csv
OK: state_count_20190203.parquet -> state_count_20190203.csv
OK: state_count_20190204.parquet -> state_count_20190204.csv
OK: state_count_20190205.parquet -> state_count_20190205.csv
OK: state_count_20190206.parquet -> state_count_20190206.csv
OK: state_count_20190207.parquet -> state_count_20190207.csv
OK: state_count_20190208.parquet -> state_count_20190208.csv
OK: state_count_20190209.parquet -> state_count_20190209.csv
OK: state_count_20190210.parquet -> state_count_20190210.csv


## 設定
- **mesh_level = 3**  
  3次メッシュ（約1km）を使用し，都市スケールの人流を表現

- **initial_poi = "home"**  
  居住地（home）から行動を開始

- **start_hour = 6**  
  シミュレーション開始時刻（朝）

- **end_hour = 18**  
  シミュレーション終了時刻（日中）

- **branch_size = 10**  
  各時刻で保持する遷移分岐の最大数

- **top_n = 3**  
  遷移確率上位のみを採用

- **validate()**  
  設定値の整合性を検証

In [ ]:
# シミュレーション設定
config = SimulationConfig(
    mesh_level=3,
    initial_poi="home",
    start_hour=6,
    end_hour=18,
    branch_size=10,
    top_n=3,
)

config.validate()

print(config)


## 初期状態の作成
- 合成人口データをロードする
- 各個人の緯度・経度を取得する
- 緯度・経度からメッシュIDを付与する
    - このメッシュIDを実験で使う空間単位とする
- 初期配置として PoI = home を与える

In [ ]:
# 合成人口データの読み込み
people_path = '/Users/y-osamu/study/poi_sim/data/processed/sakagami_data/2015_001_8_13208_01.csv' # 阪上ファイル実験ファイル

df_people = pd.read_csv(people_path, index_col=0)

rename_dict = {
    "prefecture_code": "都道府県コード",
    "prefecture_name": "都道府県名",
    "city_code": "市区町村コード",
    "city_name": "市区町村名",
    "town_code": "町丁目コード",
    "town_name": "町丁目名",
    "latitude": "緯度",
    "longitude": "経度",
    "household_id": "世帯ID",
    "family_type_id": "世帯類型ID",
    "family_type": "世帯類型",
    "num_member": "世帯人数",
    "abnormal_household": "異常世帯フラグ",
    "person_id": "個人ID",
    "age": "年齢",
    "gender_id": "性別ID",
    "gender": "性別",
    "role_household_type_id": "世帯内役割ID",
    "role_household_type": "世帯内役割",
    "industry_type_id": "産業分類ID",
    "industry_type": "産業分類",
    "employment_type_id": "就業形態ID",
    "employment_type": "就業形態",
    "company_size_id": "企業規模ID",
    "company_size": "企業規模"
}

df_people = df_people.rename(columns=rename_dict)
print("人数:", len(df_people))
df_people.head(1)

In [ ]:
states = build_initial_states(
    df_people=df_people,
    state_encoder=encoder,
    sample_n=100,
)

print(states.shape)
print(states[:10])

## 世界を作成


In [ ]:
# 初期状態
init_world = World(states)

print("個体数:", init_world.size)


## 遷移確率を用意


In [ ]:

# 遷移確率行列のあるdirを読み込み
trans_prob_mat_dir = '/Users/y-osamu/study/poi_sim/data/processed/new'

mat_files = sorted(glob.glob(os.path.join(trans_prob_mat_dir,'*.npy')))
matfile_dict = defaultdict(list)
for mat_file in mat_files:
    hour = int(os.path.basename(mat_file).split('_')[1].split('.')[0])
    mat = np.load(mat_file)
    matfile_dict[hour] = mat

print("遷移確率行列の時間帯:", sorted(matfile_dict.keys()))
for hour, mat in matfile_dict.items():
    print(f"hour={hour}, shape={mat.shape}, dtype={mat.dtype}")



In [ ]:
# 遷移行列（15分×4）
P_list = []
for _ in range(4):
    P = np.random.rand(NUM_STATE, NUM_STATE)
    P /= P.sum(axis=1, keepdims=True)
    P_list.append(P)

print(len(P_list))

# 真値（ダミー）
true_vec = np.random.rand(NUM_STATE)

print(true_vec)

In [ ]:
logger = PrintLogger()

worlds = simulate_1hour(
    init_worlds=[init_world],
    transition_matrices=P_list,
    branch_size=10,
    top_n=3,
    true_vec=true_vec,
    num_state=NUM_STATE,
    logger=logger,
)